In [2]:
import xarray as xr
import rasterio
import geopandas as gpd
import rasterstats as rstats
import numpy as np
import pandas as pd
import netCDF4 as nc

import matplotlib.pyplot as plt

In [3]:
shp_fo = '../urban_greening/msa/michiganMSA_reprojected.shp'
shp_df = gpd.read_file(shp_fo, encoding='ISO-8859-1')
shp_df = shp_df[shp_df['LSAD']=='M1']

In [3]:
from rasterio.warp import calculate_default_transform, reproject, Resampling
import rasterio

gpp_path = "../urban_greening/nov.15/michigan_test/modis-250-gpp-2015001.tif"
landcover_path = "../urban_greening/nov.15/nlcd_2016_land_cover_l48_20210604.img"

with rasterio.open(gpp_path) as src, rasterio.open(landcover_path) as ref:
    # Calculate transform and dimensions
    transform, width, height = calculate_default_transform(
        src.crs, ref.crs, ref.width, ref.height, *ref.bounds
    )
    
    aligned_meta = src.meta.copy()
    aligned_meta.update({
        "crs": ref.crs,
        "transform": transform,
        "width": width,
        "height": height,
        "nodata": src.nodata  # Use existing nodata value
    })
    
    # Reproject and write the output
    with rasterio.open("aligned_gpp.tif", "w", **aligned_meta) as dest:
        reproject(
            source=rasterio.band(src, 1),
            destination=rasterio.band(dest, 1),
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=transform,
            dst_crs=ref.crs,
            resampling=Resampling.bilinear
        )


CPLE_AppDefinedError: Too many points (10201 out of 10201) failed to transform, unable to compute output bounds.

In [3]:
landcover_path = "../urban_greening/nov.15/nlcd_2016_land_cover_l48_20210604.img"
landcover_ds = rasterio.open(landcover_path)
landcover_data = landcover_ds.read(1)

In [32]:
landcover_ds.transform

Affine(30.0, 0.0, -2493045.0,
       0.0, -30.0, 3310005.0)

In [30]:
gpp_path = "../urban_greening/nov.15/michigan_test/modis-250-gpp-2015001.tif"
gpp_ds = rasterio.open(gpp_path)

In [31]:
gpp_ds.transform

Affine(0.0020810044729201883, 0.0, -124.77494719182157,
       0.0, -0.0020810044729201883, 49.594498598633926)

In [28]:
# save resampled gpp to .tiff
import rasterio
from rasterio.enums import Resampling
from rasterio.warp import calculate_default_transform, reproject

# Define input and output file paths
input_raster = gpp_path
output_raster = 'gpp_resampled2.tif'

target = landcover_ds
target_width = target.meta['width']
target_height = target.meta['height']
target_resolution = target.transform[0]
print(target_height, target_width, target_resolution)

# Desired new resolution
new_resolution = (target_resolution, target_resolution)

with rasterio.open(input_raster) as src:
    # Calculate the transform and dimensions for the new resolution
    transform, width, height = calculate_default_transform(
        src.crs, target.crs, target.width, target.height, *target.bounds
    )

    # Update the metadata to the new resolution
    new_meta = src.meta.copy()
    new_meta.update({
        'transform': transform,
        'width': width,
        'height': height,
        "nodata": np.nan
    })

    print(new_meta)
    # Write the resampled raster to the output file
    with rasterio.open(output_raster, 'w', **new_meta) as dst:
        reproject(
            source=rasterio.band(src, 1),
            destination=rasterio.band(dst, 1),
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=target.transform,
            dst_crs=target.crs,
            resampling=Resampling.bilinear  # Use your desired resampling method
        )

104424 161190 30.0


CPLE_AppDefinedError: Too many points (10201 out of 10201) failed to transform, unable to compute output bounds.

In [8]:
resampled_gpp_path = output_raster
resampled_gpp_ds = rasterio.open(resampled_gpp_path)
resampled_gpp_data = resampled_gpp_ds.read(1)

In [33]:
gpp_ds.nodata

65535.0

In [3]:
rasterio.open("../urban_greening/gis_processed/modis/modtestmi.tif").meta

{'driver': 'GTiff',
 'dtype': 'int16',
 'nodata': 3.0,
 'width': 11408,
 'height': 20168,
 'count': 2,
 'crs': CRS.from_wkt('PROJCS["WGS_1984_Albers",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'),
 'transform': Affine(30.00000000000002, 0.0, 756109.1188588343,
        0.0, -30.000000000000046, 2691619.4366572294)}

In [6]:
rasterio.open("../urban_greening/nov.15/nlcd_2016_land_cover_l48_20210604.img").meta

{'driver': 'HFA',
 'dtype': 'uint8',
 'nodata': None,
 'width': 161190,
 'height': 104424,
 'count': 1,
 'crs': CRS.from_wkt('PROJCS["Albers Conical Equal Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["latitude_of_center",23],PARAMETER["longitude_of_center",-96],PARAMETER["standard_parallel_1",29.5],PARAMETER["standard_parallel_2",45.5],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["meters",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]'),
 'transform': Affine(30.0, 0.0, -2493045.0,
        0.0, -30.0, 3310005.0)}

In [4]:
rasterio.open("../urban_greening/nov.15/michigan_test/modis-250-gpp-2015001.tif").meta

{'driver': 'GTiff',
 'dtype': 'uint16',
 'nodata': 65535.0,
 'width': 27796,
 'height': 11889,
 'count': 2,
 'crs': CRS.from_wkt('GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'),
 'transform': Affine(0.0020810044729201883, 0.0, -124.77494719182157,
        0.0, -0.0020810044729201883, 49.594498598633926)}

In [ ]:
# gpp_path = "../urban_greening/nov.15/michigan_test/modis-250-gpp-2015001.tif"
gpp_path = "../urban_greening/gis_processed/modis/modtestmi.tif"
landcover_path = "../urban_greening/nov.15/nlcd_2016_land_cover_l48_20210604.img"

with rasterio.open(gpp_path) as src, rasterio.open(landcover_path) as ref:
    transform, width, height = calculate_default_transform(
        src.crs, ref.crs, ref.width, ref.height, *ref.bounds
    )
    aligned_meta = src.meta.copy()
    aligned_meta.update({
        "crs": ref.crs,
        "transform": transform,
        "width": width,
        "height": height,
        # "nodata": np.nan

    })
    
    # Reproject and resample the raster
    with rasterio.open('tessss.tif', "w", **aligned_meta) as dest:
        reproject(
            source=rasterio.band(src, 1),
            destination=rasterio.band(dest, 1),
            src_transform=src.transform,
            src_crs=src.crs,
            dst_transform=transform,
            dst_crs=ref.crs,
            resampling=Resampling.bilinear,
            # src_nodata=src.nodata,
            # dst_nodata=np.nan
        )

In [ ]:
from rasterio.mask import mask
output_path = 
with rasterio.open(resampled_gpp_path) as src:
    # Read shapefile
    shapefile = shp_df
    shapes = [feature["geometry"] for feature in shapefile.iterfeatures()]
    
    # Mask raster with shapefile
    out_image, out_transform = mask(src, shapes, crop=True)
    out_meta = src.meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform,
        "crs": landcover_ds.crs.to_string()
    })
    
    # Save the cropped raster
    with rasterio.open(output_path, "w", **out_meta) as dest:
        dest.write(out_image)

In [10]:
resampled_gpp_path = output_path
resampled_gpp_ds = rasterio.open(resampled_gpp_path)
resampled_gpp_data = resampled_gpp_ds.read(1)

In [12]:
gpp_path = "../urban_greening/gis_processed/modis/modis2015_rep_res.tif"
gpp_ds = rasterio.open(gpp_path)
gpp_data = gpp_ds.read(1)

In [13]:
gpp_data

array([[-32768, -32768, -32768, ..., -32768, -32768, -32768],
       [-32768, -32768, -32768, ..., -32768, -32768, -32768],
       [-32768, -32768, -32768, ..., -32768, -32768, -32768],
       ...,
       [-32768, -32768, -32768, ..., -32768, -32768, -32768],
       [-32768, -32768, -32768, ..., -32768, -32768, -32768],
       [-32768, -32768, -32768, ..., -32768, -32768, -32768]], dtype=int16)

In [11]:
resampled_gpp_data

array([[3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3],
       ...,
       [3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3]], dtype=int16)

In [ ]:
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
import geopandas as gpd
from rasterio.enums import Resampling
import numpy as np

# Paths to input files
raster_path = landcover_path  # Input raster
output_cropped_raster = "cropped_raster.tif"  # Output cropped raster
other_raster_path = gpp_path  # Raster to align
output_aligned_raster = "aligned_raster.tif"  # Output aligned raster

# Crop raster using shapefile
def crop_raster(raster_path, output_path):
    with rasterio.open(raster_path) as src:
        # Read shapefile
        shapefile = shp_df
        shapes = [feature["geometry"] for feature in shapefile.iterfeatures()]
        
        # Mask raster with shapefile
        out_image, out_transform = mask(src, shapes, crop=True)
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform,
            "crs": landcover_ds.crs.to_string()
        })
        
        # Save the cropped raster
        with rasterio.open(output_path, "w", **out_meta) as dest:
            dest.write(out_image)

# Align a raster to match another's resolution, CRS, and bounds
def align_raster(input_raster, reference_raster, output_path):
    with rasterio.open(input_raster) as src, rasterio.open(reference_raster) as ref:
        transform, width, height = calculate_default_transform(
            src.crs, ref.crs, ref.width, ref.height, *ref.bounds
        )
        aligned_meta = src.meta.copy()
        aligned_meta.update({
            "crs": ref.crs,
            "transform": transform,
            "width": width,
            "height": height
        })
        
        # Reproject and resample the raster
        with rasterio.open(output_path, "w", **aligned_meta) as dest:
            reproject(
                source=rasterio.band(src, 1),
                destination=rasterio.band(dest, 1),
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=ref.crs,
                resampling=Resampling.nearest
            )

# Perform operations
crop_raster(raster_path, output_cropped_raster)
align_raster(other_raster_path, output_cropped_raster, output_aligned_raster)

print("Cropping and alignment completed!")


Cropping and alignment completed!


In [10]:
test = rasterio.open("cropped_raster.tif")

In [13]:
test.shape

(15224, 11406)